In [348]:
%load_ext autoreload 
%autoreload 2
import os, sys
import pandas as pd 
import numpy as np 
sys.path.append('../')
from util_preprocess.fuctions import *
from matplotlib import pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Read data 

In [343]:
df = pd.read_hdf('../currency_data/currency.h5')

df['USD_NTD']  = df['美元／新台幣']
df['RMD_NTD']  = df['人民幣／新台幣']
df['UD_USD'] = df['歐元／美元']
df['USD_JPD'] = df['美元／日幣']
df['USD_RMB'] = df['美元／人民幣']
df['END_USD'] = df["英鎊／美元"]
del df['美元／新台幣']
del df['人民幣／新台幣']
del df['歐元／美元']
del df['美元／日幣']
del df['美元／人民幣']
del  df["英鎊／美元"]
df.set_index('日期', inplace=True)
df.index = pd.to_datetime(df.index)

In [349]:
df = df.sort_index()
df['USD_NTD_Trend'] = trend(list(df['USD_NTD']))
df['USD_NTD_Label'] = trend_2(list(df['USD_NTD']))
df.head()

,澳幣／美元,美元／港幣,美元／南非幣,紐幣／美元,USD_NTD,RMD_NTD,UD_USD,USD_JPD,USD_RMB,END_USD,USD_NTD_Trend,USD_NTD_Label
日期,,,,,,,,,,,,
2016-01-04,0.72130,7.75085,15.63855,NaN,33.236,5.023356,1.09265,118.855,6.6163,1.4760,0,0
2016-01-05,0.72075,7.75100,15.58535,NaN,33.270,5.009630,1.08150,119.255,6.6412,1.4713,1,1
2016-01-06,0.71020,7.75150,15.75085,NaN,33.453,4.990585,1.07355,118.635,6.7032,1.4636,2,1
2016-01-07,0.70555,7.75220,15.98500,NaN,33.530,5.012467,1.07785,118.095,6.6893,1.4593,3,1
2016-01-08,0.70340,7.75760,16.03800,NaN,33.501,5.011080,1.08930,118.265,6.6854,1.4616,2,0


In [350]:
#df['USD_NTD'].plot()
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import f1_score
df.head()
col = ["USD_NTD", "RMD_NTD","UD_USD","USD_JPD","USD_RMB","USD_NTD_Trend"]


train_df = df.copy()

In [351]:
# previous_Ndays_price([1, 2, 4, 5, 6, 7, 8], 3)
for N in [ 5, 9]:
    for c in col:
        train_df[c+'_pre_d{}'.format(N)] = previous_Ndays_price_delta( list( train_df[c]), N)
        train_df[c+'_pre_{}'.format(N)] = previous_Ndays_price( list( train_df[c]), N)
        train_df[c+'_pre_dp[{}'.format(N)] = previous_Ndays_price_delta_percent( list( train_df[c]), N)
        


In [256]:
# alone time convolution 


In [352]:
col = [i for i in train_df.columns if i not in ['USD_NTD_Label', 
                                                '紐幣／美元', 
                                                '澳幣／美元',
                                                '美元／港幣']]
display(train_df[col][9:].head())
X = train_df[col].values[9:-1]
y = train_df["USD_NTD_Label"].values[9+1:]
y_5days = up_within_Ndays(y)
y_3days = up_within_Ndays(y, 3)
y_2days = up_within_Ndays(y, 2)


,美元／南非幣,USD_NTD,RMD_NTD,UD_USD,USD_JPD,USD_RMB,END_USD,USD_NTD_Trend,USD_NTD_pre_d5,USD_NTD_pre_5,...,UD_USD_pre_dp[9,USD_JPD_pre_d9,USD_JPD_pre_9,USD_JPD_pre_dp[9,USD_RMB_pre_d9,USD_RMB_pre_9,USD_RMB_pre_dp[9,USD_NTD_Trend_pre_d9,USD_NTD_Trend_pre_9,USD_NTD_Trend_pre_dp[9
日期,,,,,,,,,,,,,,,,,,,,,
2016-01-15,16.56000,33.802,5.106570,1.08835,117.610,6.6193,1.4393,5,-0.301,33.501,...,0.003951,1.245,118.855,0.010586,-0.0030,6.6163,-0.000453,-5.0,0.0,-1.000000
2016-01-18,16.74385,33.818,5.127756,1.08835,117.245,6.5951,1.4297,6,-0.167,33.651,...,-0.006294,2.010,119.255,0.017144,0.0461,6.6412,0.006990,-5.0,1.0,-0.833333
2016-01-19,16.59670,33.820,5.124305,1.08755,117.945,6.5999,1.4300,7,-0.153,33.667,...,-0.012873,0.690,118.635,0.005850,0.1033,6.7032,0.015652,-5.0,2.0,-0.714286
2016-01-20,16.82140,33.826,5.124064,1.09505,116.725,6.6014,1.4168,8,-0.201,33.625,...,-0.015707,1.370,118.095,0.011737,0.0879,6.6893,0.013315,-5.0,3.0,-0.625000
2016-01-21,16.73580,33.838,5.117591,1.08935,117.065,6.6121,1.4167,9,-0.096,33.742,...,-0.000046,1.200,118.265,0.010251,0.0733,6.6854,0.011086,-7.0,2.0,-0.777778


In [353]:
def kflod_trainer(X, y, model, nsplit=3, feats=col):
    fold = StratifiedKFold(nsplit)
    for train_ind, valid_ind in fold.split(X, y):
        trX, trY = X[train_ind], y[train_ind]
        vaX, vaY = X[valid_ind], y[valid_ind]
        model.fit(trX, trY)
        pre_y = model.predict(vaX)
        loss = classification_report(vaY, pre_y, )
        print('----------------------------------------------------------------')
        print(loss)
        res_df = pd.DataFrame()
        res_df['feats'] = feats
        res_df.index = [round(i, 3) for i in 
                        list(model.feature_importances_)]
        display(res_df.sort_index())
for i in [2, 3, 4]:
    kflod_trainer(X, y_2days, GradientBoostingClassifier(max_depth=2 ), i)
# kflod_trainer(X, y_2days, LogisticRegression(C=0.1 ), 2)

----------------------------------------------------------------
             precision    recall  f1-score   support

        0.0       0.17      0.67      0.27        27
        1.0       0.82      0.32      0.46       127

avg / total       0.71      0.38      0.43       154



,feats
0.000,USD_NTD_Trend_pre_d9
0.000,RMD_NTD
0.000,USD_RMB_pre_d5
0.000,USD_JPD
0.000,USD_RMB
0.000,USD_RMB_pre_9
0.000,USD_JPD_pre_dp[9
0.000,USD_NTD_pre_dp[5
0.000,USD_RMB_pre_dp[5
0.000,USD_NTD_Trend_pre_dp[5


----------------------------------------------------------------
             precision    recall  f1-score   support

        0.0       0.18      0.88      0.30        26
        1.0       0.88      0.18      0.30       126

avg / total       0.76      0.30      0.30       152



,feats
0.000,USD_RMB_pre_dp[9
0.000,USD_RMB_pre_d9
0.000,USD_JPD_pre_9
0.000,USD_NTD_Trend_pre_d9
0.000,USD_NTD_Trend_pre_d5
0.000,USD_JPD_pre_5
0.000,USD_NTD_Trend
0.001,UD_USD_pre_d9
0.003,UD_USD_pre_dp[5
0.003,美元／南非幣


----------------------------------------------------------------
             precision    recall  f1-score   support

        0.0       0.21      1.00      0.35        18
        1.0       1.00      0.20      0.33        85

avg / total       0.86      0.34      0.34       103



...

,feats
0.000,UD_USD_pre_d5
0.000,USD_RMB_pre_dp[5
0.000,USD_RMB_pre_d5
0.000,UD_USD_pre_dp[9
0.000,USD_RMB
0.000,USD_JPD_pre_dp[9
0.000,USD_NTD_Trend_pre_d9
0.002,USD_JPD_pre_dp[5
0.002,UD_USD_pre_dp[5
0.002,USD_JPD_pre_d9


----------------------------------------------------------------
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00        13
        1.0       0.82      0.95      0.88        63

avg / total       0.68      0.79      0.73        76



,feats
0.000,USD_NTD_Trend_pre_d9
0.000,USD_RMB_pre_9
0.000,UD_USD_pre_dp[9
0.004,USD_NTD_Trend_pre_dp[9
0.004,USD_JPD_pre_dp[5
0.004,UD_USD_pre_5
0.005,USD_NTD_pre_9
0.006,UD_USD_pre_d9
0.006,RMD_NTD_pre_d5
0.006,UD_USD_pre_d5


----------------------------------------------------------------
             precision    recall  f1-score   support

        0.0       0.27      0.31      0.29        13
        1.0       0.85      0.83      0.84        63

avg / total       0.75      0.74      0.74        76



,feats
0.000,USD_NTD_Trend_pre_dp[9
0.000,USD_NTD_Trend_pre_d9
0.002,USD_JPD_pre_dp[5
0.002,USD_RMB_pre_9
0.003,UD_USD_pre_d9
0.004,USD_NTD_Trend_pre_d5
0.006,USD_JPD_pre_9
0.006,USD_RMB_pre_d5
0.007,USD_RMB_pre_dp[9
0.008,UD_USD_pre_dp[9


In [304]:
set(y_2days)

{1.0}

In [315]:
a = GradientBoostingClassifier()

In [ ]:
class Node(object):
    def __init__(self):
        self.l = None 
        self.r = None
        self.rule = {}
    
class DecisionTree():
    def __init__(self):
        pass
    def greedy_construct(self):
        pass
    

In [ ]:
'../sct/data/'

In [355]:
import os

import numpy
from numpy.distutils.misc_util import Configuration


def configuration(parent_package="", top_path=None):
    config = Configuration("tree", parent_package, top_path)
    libraries = []
    if os.name == 'posix':
        libraries.append('m')
    config.add_extension("_tree",
                         sources=["_tree.pyx"],
                         include_dirs=[numpy.get_include()],
                         libraries=libraries,
                         extra_compile_args=["-O3"])
    config.add_extension("_splitter",
                         sources=["_splitter.pyx"],
                         include_dirs=[numpy.get_include()],
                         libraries=libraries,
                         extra_compile_args=["-O3"])
    config.add_extension("_criterion",
                         sources=["_criterion.pyx"],
                         include_dirs=[numpy.get_include()],
                         libraries=libraries,
                         extra_compile_args=["-O3"])
    config.add_extension("_utils",
                         sources=["_utils.pyx"],
                         include_dirs=[numpy.get_include()],
                         libraries=libraries,
                         extra_compile_args=["-O3"])

    config.add_subpackage("tests")

    return config


from numpy.distutils.core import setup
setup(**configuration().todict())

non-existing path in '': '_tree.pyx'
non-existing path in '': '_splitter.pyx'
non-existing path in '': '_criterion.pyx'
non-existing path in '': '_utils.pyx'


ValueError: 'tests' is not a directory